<a href="https://colab.research.google.com/github/shuwang127/636Project/blob/main/Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np

_COLAB_ = 0 if (os.getenv('COLAB_GPU', 'NONE') == 'NONE') else 1
rootPath = './drive/My Drive/Colab Notebooks/' if (_COLAB_) else './'

# read data.
traindata = pd.read_csv(rootPath + 'training_data.csv')
testdata = pd.read_csv(rootPath + 'testing_data.csv')

# fill missing features.
traindata = traindata.replace(' ?', np.nan)
for index, row in traindata.iteritems():
    traindata[index].fillna(traindata[index].mode()[0], inplace=True)
testdata = testdata.replace(' ?', np.nan)
for index, row in testdata.iteritems():
    testdata[index].fillna(testdata[index].mode()[0], inplace=True)

In [2]:
# get labels.
YTrain = traindata['salary'].replace([' <=50K' , ' >50K'], [0, 1])

# get features.
XTrain = traindata.drop(['ID (this is not a feature)', 'salary'], axis=1)
XTest = testdata.drop(['ID (this is not a feature)'], axis=1)
# normalize numeric features.
from sklearn.preprocessing import StandardScaler
numcol = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
scaler = StandardScaler()
XTrain[numcol] = scaler.fit_transform(XTrain[numcol])
XTest[numcol] = scaler.transform(XTest[numcol])
# get one-hot vectors.
X = pd.concat([XTrain,XTest])
X = pd.get_dummies(X)
XTrain = X[:len(XTrain)]
XTest = X[len(XTrain):]
print("size of training data is", XTrain.shape)
print("size of testing data is", XTest.shape)

size of training data is (38842, 105)
size of testing data is (10000, 105)


In [3]:
# Neural Network.
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(32,8), activation='logistic', solver='lbfgs', max_iter=100)
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [4]:
testdata['pred-salary'] = YPred
testdata['pred-salary'] = testdata['pred-salary'].replace([0, 1], [' <=50K' , ' >50K'])
result = testdata[['ID (this is not a feature)', 'pred-salary']]
result.to_csv(rootPath + 'result.csv', index=0)
result.head()

,ID (this is not a feature),pred-salary
0,1,<=50K
1,2,<=50K
2,3,>50K
3,4,<=50K
4,5,<=50K
